In [1]:
env_USE_GOOGLE_COLAB = False
env_USE_NVIDIA = False
env_CONTENT_PATH = ""
env_TIF_PATH = ""
env_WANT_NGROK = False
env_Latest_RUN_ID = 0
import sys
from settings import load_BatGutsSettings, show_python_version, remove_matching_files, get_most_recent_subdirectory, get_most_recent_numeric_subdirectory
S = load_BatGutsSettings()
env_TIF_PATH = S.zTifPath
env_CONTENT_PATH = S.zContentPath
env_USE_GOOGLE_COLAB = S.useGoogleColab
env_USE_NVIDIA = S.useNvidia
if len(env_CONTENT_PATH) == 0:
    sys.exit("Exiting the script because setings were not processed")

Loading settings from /Users/thomasyohe/isr/BatGutsImageClassification2/jupyternb/.env
my_CONTENT_PATH=/Users/thomasyohe/isr/bguts
my_TIF_PATH=/Users/thomasyohe/isr/bguts/tiffs


In [2]:
if env_USE_GOOGLE_COLAB:
    from google.colab import drive
    drive.mount('/content/drive/')

In [3]:
import numpy as np
from matplotlib import pyplot as plt
import tifffile as tiff

from pyngrok import ngrok
import numpy as np
from PIL import Image as im
from PIL import  ImageOps
print( "PIL Version="+str(im.__version__))

PIL Version=10.2.0


In [4]:
# Import necessary libraries for deep learning and data manipulation

import numpy as np  # Library for numerical operations
import tensorflow as tf  # Library for building machine learning models
from tensorflow import keras  # High-level API for Keras functionality
from tensorflow.keras.models import Sequential  # Type of neural network model
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout  # Various layers for building models

# Import libraries for training and optimizing models

from tensorflow.keras.optimizers import Adam  # Optimization algorithm for training
from tensorflow.keras.metrics import categorical_crossentropy  # Metric for evaluating multi-class classification

# Import libraries for data augmentation and preprocessing

from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Library for image augmentation
from tensorflow.keras.applications.resnet50 import ResNet50  # Pre-trained model for image classification

# Import libraries for saving and loading models, callbacks, and metrics

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau  # Callbacks for model optimization
from tensorflow.keras.models import Model  # Type of neural network model
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report  # Metrics for evaluating model performance
from tensorflow.keras.utils import to_categorical  # Function for converting labels to categorical format

# Import libraries for file manipulation and plotting

import itertools  # Library for combinatorial algorithms
import os  # Library for interacting with the operating system
import shutil  # Library for copying and moving files
import random  # Library for generating random numbers
import glob  # Library for finding files based on patterns
import matplotlib.pyplot as plt  # Library for creating static, animated, and interactive visualizations
import warnings
from tensorflow.python.keras.utils.data_utils import Sequence

# Suppress future warnings from TensorFlow

warnings.simplefilter(action='ignore', category=FutureWarning)

# Display the version of TensorFlow being used
print("Tensorflow version=" + str(tf.__version__))

# Enable inline plotting with Matplotlib
%matplotlib inline


Tensorflow version=2.16.1


In [5]:
import mlflow

In [6]:
zUrl = env_CONTENT_PATH + "/mlruns/"
mlflow.set_tracking_uri(zUrl)

In [7]:
zPath = env_CONTENT_PATH + "/mlruns"
remove_matching_files( zPath, "*")


In [8]:
EXPERIMENT_NAME = "Batguts-Insects_vs_plants_Unet_256"
EXPERIMENT_ID = mlflow.create_experiment(EXPERIMENT_NAME)

MlflowException: Experiment 'Batguts-Insects_vs_plants_Unet_256' already exists.

In [ ]:
"""
This code defines a simple function called `display_image` that displays an image from its original file path.
The function takes one argument, `path`, which is expected to be the file path of the TIFF image to be displayed.
Within the function, it uses the `tiff.imread()` function (from the `tifffile` library) to read the image data 
from the specified file path and stores it in a variable called `baseimage`.
The original shape of the image is then printed out for reference purposes.
Next, any unnecessary dimensions are removed from the image data using `np.squeeze()`, which helps simplify plotting.
Finally, a new figure is created for displaying the image using `plt.figure()`, 
and the image itself is displayed within this figure using `plt.imshow()`.

"""


# Define a function to display an image at its original path.

def display_image(path):
    """
    This function displays an image from the given path.

    Args:
        path (str): The file path of the image to be displayed.
    """

    # Read the TIFF image from the specified path using tiff.imread

    baseimage = tiff.imread(path)

    # Print the original shape of the image for reference

    print('Original image shape:', baseimage.shape)

    # Remove any unnecessary dimensions (i.e., squeeze) to make plotting easier

    baseimage = np.squeeze(baseimage)

    # Create a new figure in which to display the image using plt.figure()

    plt.figure()

    # Display the image within the newly created figure using plt.imshow()
    
    plt.imshow(baseimage)


In [ ]:
zBase = env_TIF_PATH + "/LS042_Cormura_brevirostris-LY20_10-insect"
zDir = zBase + "/LS042_Cormura_brevirostris-LY20_10-insect-guts-LY20-10A-AB-08"
zFile = zDir + "/LS042_Cormura_brevirostris-LY20_10-insect-guts-LY20-10A-AB-08-Plane000.tif"
display_image(zFile)

In [ ]:
zBase = env_TIF_PATH + "/DRXXX_Brachyphylla_pumila-LY20_13-nectar"
zDir = zBase + "/DRXXX_Brachyphylla_pumila-LY20_13-nectar-guts-LY20-13A-HE-006"
zFile = zDir + "/DRXXX_Brachyphylla_pumila-LY20_13-nectar-guts-LY20-13A-HE-006-Plane004.tif"
display_image(zFile)

In [ ]:
"""
Import Seaborn library for creating informative and attractive statistical graphics.
Define a function called `standardize_image` that takes in a list of images as input.
"""

import seaborn as sns  # 'as' keyword assigns an alias, 'sns', to the library for easy use

# Import OpenCV (cv2) library for image and video processing tasks.

import cv2  # This library provides a wide range of functionalities for image and video analysis.

# Import time library to measure execution times or timestamps in your code.

import time  # This library is used to track the passage of time, which can be useful in various scenarios.

# Import os (operating system) library to interact with the file system and perform various operations.

import os  # This library provides a way to create, delete, and manage directories and files.

# Import glob library for finding files based on patterns or globbing.

import glob  # This library is used to search for files in directories and subdirectories that match specific criteria.


In [ ]:
#Standardizing the image

def standardize_image(img_list):
  # Calculate the mean image by taking the average of all images along the axis=0 (i.e., feature-wise).
  mean_img = np.mean(img_list, axis=0)
  
  # Calculate the standard deviation of each image by taking the std along the axis=0 (i.e., feature-wise).
  std_img = np.std(img_list, axis=0)
  
  # Subtract the mean from each image and then divide by the standard deviation to standardize the images.
  img_list = (img_list - mean_img) / std_img
  
  # Return the list of standardized images.
  return img_list


In [ ]:
# Define a function to train images based on their categories.

   
def train_imgs(category):
    img_list = []
    label_list = []
    category_path = os.path.join(env_CONTENT_PATH, "images/Bats_mixed_images_RGB_256", str(category))
    if not os.path.exists(category_path):
        print(f"Directory does not exist: {category_path}")
        return img_list, label_list
    for main_path in glob.glob(category_path + "/"):
        paths = glob.glob(os.path.join(main_path, "*.jpeg"))
        print('{} Images Count: {}'.format(category, len(paths)))
        for image_path in paths:
            image = cv2.imread(image_path)
            if image is not None:
                img_list.append(image)
                label_list.append(category)
            else:
                print(f"Failed to read image: {image_path}")
    print(f'{category} images loaded: {len(img_list)}, labels loaded: {len(label_list)}')
    return img_list, label_list



In [ ]:

insects_list, insect_labels = train_imgs('Insects')
plants_list, plant_labels = train_imgs('Plants')

print(f'Total insects images: {len(insects_list)}, Total plants images: {len(plants_list)}')

train_imgs_list = insects_list + plants_list
labels_list = insect_labels + plant_labels

In [ ]:
len(plants_list[0:97])

In [ ]:
# Import the LabelEncoder class from scikit-learn's preprocessing module
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
labels_list = label_encoder.fit_transform(labels_list)

# Note: The output is not a simple numerical assignment, but rather an array-like object with the transformed labels
# Ensure lengths match

assert len(train_imgs_list) == len(labels_list), "The number of images and labels should match"

In [ ]:
# Import the Counter class from Python's built-in collections module
from collections import Counter

# Create a Counter object to count the occurrences of each label in the list
counter = Counter(labels_list)

# Print or store the resulting Counter object, which contains the frequency of each label
print(counter)

In [ ]:

"""
This section of the code shuffles the combined train images and labels into a random order.
The goal is to randomize the data so that machine learning models are trained on a diverse set of examples, 
which can help improve their generalization performance.
"""
#Shuffle the data

import random

temp = list(zip(train_imgs_list, labels_list))
random.shuffle(temp)
if len(temp) == 0:
    print("No data to unpack. Check if images are being loaded correctly.")
train_imgs_list, labels_list = zip(*temp)
train_imgs_list, labels_list = list(train_imgs_list), list(labels_list)


In [ ]:
len(train_imgs_list)

In [ ]:
len(labels_list)

In [ ]:
# Convert to numpy arrays and preprocess

X = np.array(train_imgs_list, np.float32) / 255.0
X = standardize_image(X)
labels_list = np.array(labels_list)
# Ensure lengths match
assert X.shape[0] == labels_list.shape[0], "The number of images and labels should match after preprocessing"


In [ ]:
X.shape

In [ ]:
len(X)

In [ ]:
"""
Converting List of Labels to NumPy Array

# In many machine learning applications, it's common to work with lists or arrays of labels. 
# Here's how you can convert a list into a numpy array.

# Why Use NumPy Arrays?

# NumPy arrays are powerful data structures that can significantly improve performance and efficiency in numerical computations. 
# By using numpy arrays, you can take advantage of vectorized operations, which can speed up your code by orders of magnitude compared 
# to using Python lists or other data structures.

"""

labels_list = np.array(labels_list)

# This line of code converts the list of labels into a numpy array.
# The resulting array has the same elements as the original list, but with additional capabilities and features provided by numpy.

In [ ]:
Counter(labels_list)

In [ ]:
len(labels_list)

In [ ]:
# X_smote.shape

In [ ]:
# X_smote.shape

In [ ]:
np.array(labels_list).shape

In [ ]:


"""
The function one_hot_labels(img_list, classes) generates one-hot encoded labels from a list of image labels.

# Here's how it works:

# It creates a dictionary label_numeric where each unique label in img_list 
# is assigned a unique numeric value. It then replaces the original labels with their corresponding 
# numeric values using a list comprehension (target_values = [label_numeric[k] for k in img_list]). 
# The resulting list of numeric labels is converted to an numpy array (target_values = np.array(target_values)). 
# Finally, it uses the to_categorical() function (which likely comes from the Keras library) 
# to generate one-hot encoded labels from the numeric labels. 
# The num_classes=2 argument means that there are only 2 possible classes or categories in the data. 
# The to_categorical() function will output a numpy array with shape (n_samples, 2), 
# where each row represents a sample and has two elements: one for class 0 and one for class 1.
"""

# This line of code specifies the number of classes or categories that the model will be trained on.

num_classes = 2

def one_hot_labels(img_list,classes):
  label_numeric = {k:v for v,k in enumerate(set(img_list))}
  target_values = [label_numeric[k] for k in img_list]
  target_values = np.array(target_values)
  target_values = to_categorical(target_values,num_classes=num_classes)
  return target_values

labels_list = one_hot_labels(labels_list,num_classes)

In [ ]:
labels_list.shape

In [ ]:
labels_list = np.array(labels_list)

In [ ]:
len(labels_list)

In [ ]:
# Splitting the data into train and validation set

# This line of code is using a Python syntax to import a specific function or module. 
# In this case, it's importing the train_test_split function, which will be used later in the code.

from sklearn.model_selection import train_test_split

print("Shape of X:", X.shape)
print("Shape of labels_list:", labels_list.shape)

""" 
First Train-Test Split

# The next block of code splits the data into training and testing sets.
# Here's a breakdown of what this line does:

# train_test_split: This is the function being called to split the data.
# X_smote and labels_list_smote: These are the input data and labels that need to be split into training and testing sets.
# test_size=0.2: This parameter specifies that 20% of the data should be used for testing, while the remaining 90% is used for training.
# random_state=1: This parameter ensures that the same random seed is used every time the function is called, which helps with reproducibility.
# The result of this line of code is four new variables: X_train, x_test, Y_train, and y_test. 
# These variables represent the training data (X_train), testing data (x_test), training labels (Y_train), and testing labels (y_test).
"""

X_train, x_test, Y_train, y_test = train_test_split(X, labels_list, test_size=0.2, random_state=1, stratify=labels_list) # 20% of the data for validation + test

"""
Second Train-Valid Split

The final block of code splits the training data into two subsets: one for validation and one for further training.

This line of code is similar to the first one, but with a few key differences:

X_train and Y_train: These are the input data and labels that need to be split into validation and further training sets.
test_size=0.5: This parameter specifies that 50% of the data should be used for validation, 
while the remaining 50% is used for further training.
The variables x_train, x_valid, y_train, and y_valid are created to represent the further training data (x_train), 
validation data (x_valid), further training labels (y_train), and validation labels (y_valid).
"""
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, test_size=0.5, random_state=1, stratify=Y_train)  # Split the 20% equally for validation and test


In [ ]:
# Debugging step to verify the number of images and labels after splitting
print(f'Train images: {X_train.shape[0]}, Train labels: {Y_train.shape[0]}')
print(f'Validation images: {x_valid.shape[0]}, Validation labels: {y_valid.shape[0]}')
print(f'Test images: {x_test.shape[0]}, Test labels: {y_test.shape[0]}')

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
# Printing lengths and shapes
lengths_shapes = {
    "Length of train_imgs_list": len(train_imgs_list),
    "Length of labels_list": len(labels_list),
    "Shape of X": X.shape,
    "Shape of labels_list": labels_list.shape,
    "Shape of x_train": x_train.shape,
    "Shape of y_train": y_train.shape,
    "Shape of x_valid": x_valid.shape,
    "Shape of y_valid": y_valid.shape,
    "Shape of X_train": X_train.shape,
    "Shape of Y_train": Y_train.shape,
    "Shape of x_test": x_test.shape,
    "Shape of y_test": y_test.shape
}


In [ ]:
# Displaying lengths and shapes using pandas
import pandas as pd
lengths_shapes_df = pd.DataFrame(lengths_shapes.items(), columns=['Variable', 'Value'])
print(lengths_shapes_df)

In [ ]:
"""
Explanation of train_datagen.fit(X)

Overview
The provided code snippet involves using a Keras data generator, specifically a DataGenerator object named train_datagen, to fit the training data. 
The main function being called is fit(), which is used to initialize and configure the data generator for training.

Step 1: Understanding the fit() Method
The fit() method is a common interface in Keras that allows you to train a model with a given dataset. 
In this case, we're using it to fit the training data to the train_datagen object.

Here's what's happening:

train_datagen: This is an instance of a Keras DataGenerator, which is used to generate batches of data for training.
.fit(): This method is called on the train_datagen object, passing in the training data (X) as input.
Step 2: Breaking Down the fit() Method
The fit() method takes several parameters that are used to configure the data generator. Some of these parameters include:

X: The input training data.
y: Not specified here; it's assumed to be provided separately (e.g., in a separate code snippet).
epochs: The number of epochs to run during training.
verbose: A flag indicating whether to display progress updates during training.
validation_data: An optional parameter for providing validation data.
In this specific case, we're only passing in the input data (X) without specifying other parameters.
This implies that the default values will be used for any unspecified parameters.

Step 3: What's Happening Under the Hood
When you call fit() on a Keras data generator, several things happen:

Data initialization: The data generator is initialized with the provided input data (X).
Model configuration: The model architecture and training settings are configured based on the DataGenerator instance.
Batch generation: Batches of data are generated from the input data according to the specified batch size.
The resulting output will depend on the specific configuration of the data generator, including the number of epochs, batch size, and other parameters.
"""
train_datagen = ImageDataGenerator(
        rotation_range=10, # rotation
        width_shift_range=0.2, # horizontal shift
        height_shift_range=0.2, # vertical shift
        zoom_range=0.2, # zoom
        horizontal_flip=True, # horizontal flip
        brightness_range=[0.2,1.2],# brightness
        fill_mode='nearest')

train_datagen.fit(X)

x_train.shape

In [ ]:
"""
Explanation of base_model = ResNet50(include_top=False,weights='imagenet',input_shape=(256,256,3))

Overview
The provided code snippet involves creating a pre-trained ResNet50 model with specified parameters. This model is used as the base for further training or fine-tuning.

Breakdown of Parameters

ResNet50: This line creates an instance of the ResNet50 model architecture from Keras.
include_top=False: By setting this parameter to False, we're telling the model not to include the 
top layers (i.e., the classification head) when creating the base model. 
This is useful for using the pre-trained weights without any additional overhead.
weights='imagenet': This parameter specifies that we want to load pre-trained weights from the ImageNet dataset. 
These weights are trained on a large-scale image classification task and can be used as a starting point for our own tasks.
input_shape=(256, 256, 3): This line sets the input shape of the model to images with dimensions 256x256 and 3 color channels (RGB).

Step-by-Step Explanation

Creating the ResNet50 Model: The first line, ResNet50, creates an instance of the ResNet50 model architecture.
Removing Top Layers: By setting include_top=False, we remove the top layers from the model, which typically consist of a classification head and possibly some additional fully connected layers.
Loading Pre-Trained Weights: The line weights='imagenet' loads pre-trained weights from the ImageNet dataset into our ResNet50 instance.
Specifying Input Shape: Finally, we set the input shape to 256x256 images with 3 color channels (RGB) using the line input_shape=(256, 256, 3).
By combining these parameters, we create a pre-trained ResNet50 model without any additional overhead or top layers. This can be used as a starting point for our own image classification tasks.

Example Use Case

This code snippet is useful when you want to use the benefits of a pre-trained model (such as knowledge distillation) while also having some flexibility in terms of input shapes and architectures. You could then fine-tune this model on your own dataset or use it as a starting point for further training.

For example, you might use this code to create a base model for an image classification task.

Then, you could add additional layers on top of the base_model to create your own architecture.
"""
# base_model = ResNet50(include_top=False,weights='imagenet',input_shape=(256,256,3))


In [ ]:
"""
Code Explanation
Importing Necessary Libraries
The provided code starts with importing a necessary library.

from keras_unet.models import vanilla_unet
This line imports the vanilla_unet function from the keras_unet.models module.
The keras_unet.models module is likely a custom or external library that provides pre-built models for various tasks, including image segmentation and classification.
Creating a Base Model
The next block of code creates an instance of the vanilla_unet model with a specified input shape.

base_model = vanilla_unet(input_shape=(256, 256, 3))
This line calls the vanilla_unet function to create a new instance of the model.
The input_shape=(256, 256, 3) parameter specifies the expected input shape for this model. In this case, it's expecting images with dimensions 256x256 and 3 color channels (RGB).
Step-by-Step Explanation
Here's a breakdown of what's happening in the code:

Importing the Library: The first line imports the necessary library (keras_unet.models) that provides the vanilla_unet function.
Creating an Instance of the Model: The second line calls the vanilla_unet function to create a new instance of the model.
Specifying Input Shape: The input_shape=(256, 256, 3) parameter specifies the expected input shape for this model.
Purpose and Context
The purpose of this code is likely to use the vanilla_unet model as a starting point for further training or fine-tuning on a specific image segmentation task. The input shape specified in the code (256x256 images with 3 color channels) suggests that the model will be used for tasks involving images of similar dimensions.

Example Use Cases
This code can be useful in various scenarios, such as:

Image segmentation: Using the vanilla_unet model as a starting point for fine-tuning on specific image segmentation tasks.
Image classification: Adapting the vanilla_unet model to perform image classification tasks by adding additional layers or modifying the architecture.
Transfer learning: Using the pre-trained weights of the vanilla_unet model as a starting point for training on a new task.

"""
from keras_unet.models import vanilla_unet

base_model  = vanilla_unet(input_shape=(256, 256, 3))

In [ ]:
"""
Code Explanation
Importing Necessary Libraries
The provided code starts with importing a necessary library.

from keras.models import Sequential
This line imports the Sequential class from the keras.models module.
The keras.models module is part of the Keras deep learning framework, which provides various tools and classes for building neural networks.
The Sequential class is a simple way to build neural networks by stacking layers on top of each other.
Step-by-Step Explanation
Here's a breakdown of what's happening in this line:

Importing the Library: This line imports the necessary library (keras.models) that provides the Sequential class.
Accessing the Class: By using from keras.models import Sequential, we're accessing the Sequential class directly, so we can use it without having to prefix it with the full module name.
Purpose and Context
The purpose of this line is to prepare for building a neural network model using the Keras framework. The Sequential class will be used as a base for creating a sequence of layers that can be stacked on top of each other to build a complex neural network architecture.

Example Use Cases
This code can be useful in various scenarios, such as:

Building simple neural networks: Using the Sequential class as a starting point for building simple neural networks.
Stacking layers: Creating a sequence of layers using the Sequential class and stacking them on top of each other to build complex neural network architectures.
Next Steps

Once this line is executed, you can proceed with defining the architecture of your neural network model by adding layers to the Sequential instance. For example:

model = Sequential()
# Add layers to the model...

This will allow you to specify the structure and parameters of your neural network, which can then be trained on your dataset using Keras's training tools.

"""
from keras.models import Sequential


In [ ]:
"""
Detailed Explanation of the Provided Code
Importing Necessary Libraries
The first line imports a necessary library

from tensorflow.keras.layers import BatchNormalization

This line imports the BatchNormalization class from the tensorflow.keras.layers module.
The BatchNormalization layer is used to normalize the inputs of a neural network, which can help improve the stability and performance of the model.

Creating a Sequential Model
The next line creates a sequential model:

add_model = Sequential()

This line creates an instance of the Sequential class, which is a simple way to build neural networks by stacking layers on top of each other.
The Sequential model will be used as a building block for the final neural network architecture.
Adding Layers to the Model
The following lines add layers to the sequential model:

add_model.add(Flatten(input_shape=base_model.output_shape[1:]))

This line adds a Flatten layer to the model.
The Flatten layer is used to flatten the output of a convolutional neural network (CNN) or other layered structure into a 1D array.
The input_shape parameter specifies the shape of the input data, which in this case is taken from the output of the base_model.

add_model.add(Dense(256,activation='relu'))

This line adds a dense (fully connected) layer to the model with 256 neurons and ReLU activation.
The Dense layer is used to create a fully connected neural network where each neuron in one layer connects to every neuron in another layer.
The activation parameter specifies the activation function to be applied to the output of the layer.

add_model.add(Dropout(0.3))

This line adds a dropout layer to the model with a dropout rate of 0.3.
The Dropout layer is used to randomly drop (or "drop out") neurons during training, which can help prevent overfitting and improve the generalization of the model.

add_model.add(Dense(256,activation='relu'))

This line adds another dense layer with 256 neurons and ReLU activation.
The purpose of this second dense layer is to further process and transform the input data before passing it through the final output layer.

add_model.add(Dropout(0.2))

This line adds a dropout layer with a dropout rate of 0.2.
The purpose of this second dropout layer is to help prevent overfitting by randomly dropping neurons during training.

add_model.add(Dense(1,activation='sigmoid'))

This line adds a final dense layer with a single output neuron and sigmoid activation.
The Dense layer is used to create a fully connected neural network where each neuron in one layer connects to every neuron in another layer.
The activation parameter specifies the activation function to be applied to the output of the layer, which in this case is the sigmoid function.

Creating the Final Model
The next line creates the final model by combining the base model and the additional layers:

model = Model(inputs=base_model.input,outputs = add_model(base_model.output))

This line creates a new instance of the Model class, which is used to represent the entire neural network architecture.
The inputs parameter specifies the input data for the model, which in this case is taken from the input of the base model.
The outputs parameter specifies the output data for the model, which in this case is generated by passing the output of the base model through the additional layers.

Displaying Model Summary
The final line displays a summary of the model:


model.summary()

This line calls the summary method on the model instance to display a concise summary of its architecture and parameters.

"""


from tensorflow.keras.layers import BatchNormalization
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256,activation='relu'))
add_model.add(Dropout(0.3))
add_model.add(Dense(256,activation='relu'))
add_model.add(Dropout(0.2))
add_model.add(Dense(2,activation='softmax'))
model = Model(inputs=base_model.input,outputs = add_model(base_model.output))
model.summary()

In [ ]:
# from keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import plot_model

In [ ]:
plot_model(model, show_shapes=True, to_file='Vanila_unet_TL.png')

In [ ]:
# Compiling Model
optimizer = Adam(learning_rate=1e-5)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])




In [ ]:
# Defining callback Methods
"""
Early Stopping and Learning Rate Reduction
The provided code snippet is used to implement two important techniques in deep learning: Early Stopping and Learning Rate Reduction.

Early Stopping
Early stopping is a technique used to prevent overfitting by stopping the training process when the model's performance on a validation set starts to degrade.

early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1,
                           mode='auto', restore_best_weights=True)

EarlyStopping: This is the Keras callback function used for early stopping.
monitor='val_loss': The metric that will be monitored. In this case, it's the validation loss.
patience=10: The number of epochs to wait before applying early stopping. 
If the model's performance doesn't improve within this number of epochs, the training process is stopped.
verbose=1: This parameter controls the verbosity level of the callback function. 
A value of 1 means that the callback will print messages during execution.
mode='auto': The mode in which to apply early stopping. In this case, it's set to 'auto', 
which means that the callback will automatically choose the best weights from the previous epochs when the training process is stopped.
restore_best_weights=True: If the model's performance doesn't improve within the patience period, 
this parameter specifies whether to restore the best weights from the previous epoch.

Learning Rate Reduction
Learning rate reduction is another technique used to prevent overfitting. It involves reducing the learning rate when the model's performance on a validation set starts to degrade.


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5,
                             verbose=1, mode='auto')

ReduceLROnPlateau: This is the Keras callback function used for learning rate reduction.
monitor='val_loss': The metric that will be monitored. In this case, it's the validation loss.
factor=0.2: The factor by which to reduce the learning rate when the model's performance on the validation set degrades.
patience=5: The number of epochs to wait before reducing the learning rate. 
If the model's performance doesn't improve within this number of epochs, the learning rate is reduced.
verbose=1: This parameter controls the verbosity level of the callback function. A value of 1 means that the callback will print messages during execution.
mode='auto': The mode in which to apply learning rate reduction. In this case, it's set to 'auto', 
which means that the callback will automatically choose the best weights from the previous epochs when the training process is stopped.

n_epoch = 60

This line sets the number of epochs for which the model will be trained. In this case, it's set to 60. 
The EarlyStopping and ReduceLROnPlateau callback functions are used in conjunction with this parameter to prevent overfitting during training.

Note that the actual number of epochs may vary depending on the specific problem being tackled and the performance metrics used.

"""
n_epoch = 60

early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1,
                           mode='auto', restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5,
                             verbose=1, mode='auto')

In [ ]:
"""
Detailed Explanation of the Provided Code
Importing Necessary Modules
The provided code snippet imports necessary modules from scikit-learn:


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

This line imports four metrics from the sklearn.metrics module: accuracy_score, precision_score, recall_score, and f1_score.
These metrics are used to evaluate the performance of machine learning models.
Understanding the Metrics
Let's break down each metric:

Accuracy Score
The accuracy score is a simple measure of how well a model performs on a given dataset. 
It calculates the proportion of correctly classified instances out of all instances in the test set.


accuracy_score(y_true, y_pred)

y_true is the true labels (ground truth) of the data.
y_pred is the predicted labels by the model.

Precision Score
The precision score measures the proportion of correctly classified instances among all positive predictions made by a model. 
In other words, it tells us how often the model's positive predictions are correct.

precision_score(y_true, y_pred)

y_true is the true labels (ground truth) of the data.
y_pred is the predicted labels by the model.

Recall Score
The recall score measures the proportion of correctly classified instances among all actual positive instances in the test set. 
In other words, it tells us how often a model's predictions are correct when they should be positive.


recall_score(y_true, y_pred)

y_true is the true labels (ground truth) of the data.
y_pred is the predicted labels by the model.

F1 Score
The F1 score is a weighted average of precision and recall. It provides a balanced measure of both precision and recall, 
with more importance given to accuracy when classifying instances as positive or negative.


f1_score(y_true, y_pred)

y_true is the true labels (ground truth) of the data.
y_pred is the predicted labels by the model.

Using These Metrics in Practice

These metrics can be used to evaluate the performance of machine learning models on a test set. 
By calculating these scores, you can get an idea of how well your model performs compared to other models or baseline models.
For example, if you're building a classification model for binary classification problems (e.g., spam vs non-spam emails), 
you might use accuracy score, precision score, and recall score to evaluate the performance. 
The F1 score would be useful when there's an imbalance in class sizes.
Remember that these metrics are not always perfect and should be used in 
conjunction with other evaluation techniques to get a complete picture of your model's performance.

"""

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [ ]:
"""
Detailed Explanation of the Provided Code
Using TensorFlow's Eager Execution Mode
The provided code snippet sets up TensorFlow's execution mode to run functions eagerly:

tf.config.run_functions_eagerly(True)

This line uses the tf.config API to set the execution mode for TensorFlow.
The run_functions_eagerly() function takes a boolean value as input, which determines whether to run functions eagerly or not.

What is Eager Execution?
In traditional TensorFlow, operations are executed in a graph-based manner. 
This means that when you create a model and perform computations on it, the operations are recorded in a graph data structure. 
Then, at some point (e.g., during training), this graph is executed to produce the desired output.
However, when using Eager Execution, TensorFlow executes operations immediately as they are created. 

This provides several benefits:

Faster Development: With Eager Execution, you can see the results of your model's computations in real-time, which makes it easier to develop and debug models.
Simplified Debugging: Since operations are executed immediately, it's simpler to understand what's happening inside your model.
Better Support for Custom Operations: Eager Execution allows TensorFlow to execute custom operations, such as those defined using the tf.function API.

Step-by-Step Breakdown
Here's how this line of code works:

The first part tf.config accesses the configuration module within TensorFlow.
The run_functions_eagerly() function is called on this configuration object, which takes a boolean value as input.
In this case, the boolean value passed to the function is True, which tells TensorFlow to run functions eagerly.
By setting this flag to True, you're telling TensorFlow to execute operations immediately, rather than recording them in a graph for later execution.

Note that when running on GPU devices (e.g., NVIDIA GPUs), it's generally recommended to keep the default graph-based mode for performance reasons. 
However, when developing and debugging models, Eager Execution can be quite useful!

"""
tf.config.run_functions_eagerly(True)

In [ ]:
# Printing shapes to verify
print(f'x_train shape: {x_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'x_valid shape: {x_valid.shape}')
print(f'y_valid shape: {y_valid.shape}')
print(f'x_test shape: {x_test.shape}')
print(f'y_test shape: {y_test.shape}')

In [ ]:
y_train

In [ ]:
x_train

In [ ]:
# Ensure lengths match
assert x_train.shape[0] == y_train.shape[0], "The number of training images and labels should match"
assert x_valid.shape[0] == y_valid.shape[0], "The number of validation images and labels should match"
assert x_test.shape[0] == y_test.shape[0], "The number of test images and labels should match"

In [ ]:
"""
Detailed Explanation of the Provided Code

history = model.fit(train_datagen.flow(x_train, y_train,
                                         batch_size=10),
                    epochs=n_epoch,
                    callbacks=[reduce_lr,early_stop],
                    validation_data=(x_valid,y_valid))
                    
Step 1: Defining the Training Data Generator

train_datagen.flow(x_train, y_train, batch_size=10)

This line defines a data generator using Keras' flow() method.
The first argument x_train is the input data for training.
The second argument y_train is the corresponding labels or outputs.
The optional third argument batch_size=10 specifies that we want to process 10 samples at a time in batches.
The purpose of using a generator here is to avoid loading all the training data into memory at once. 
This can be particularly useful when working with large datasets, as it allows us to train the model incrementally and more efficiently.

Step 2: Defining the Model Fitting Functionality

history = model.fit(...)

Here we're calling the fit() method on our Keras model instance.
The first argument is the training data generator defined in Step 1, 
which will provide batches of input and output data for each iteration during training.
The optional arguments specify various hyperparameters that control how the model trains.

Step 3: Specifying Training Epochs

epochs=n_epoch,

This line specifies the number of epochs (i.e., passes through the entire dataset) to use when training the model. 
In this case, we're letting it equal n_epoch, which is presumably defined elsewhere in our code.
The idea behind epochs is that each iteration will update the weights of the model based on its performance on a small batch of data.

Step 4: Defining Callbacks for Early Stopping and Learning Rate Reduction

callbacks=[reduce_lr,early_stop],

Here we're passing a list of callback functions to the fit() method.
These callbacks are used to monitor the training process and take action when certain conditions are met.
In this case, we have two callbacks: reduce_lr (learning rate reduction) and early_stop. 
The first one will reduce the learning rate after each epoch based on a specific schedule. This can help the model converge more smoothly by adjusting its step size adaptively. The second one (early_stop) will stop training when the performance of the model on the validation set doesn't improve for a certain number of epochs (which we'll discuss next).

Step 5: Specifying Validation Data

validation_data=(x_valid,y_valid)

This line specifies the validation data to be used during training. 
We're passing in x_valid and y_valid, which are presumably defined elsewhere in our code.
The idea behind this is that we want to monitor how well the model performs on 
unseen data (the validation set) as it trains on the entire dataset.
The validation data will be passed through the network after each epoch of training, 
allowing us to assess its performance without affecting its learning process. This can help prevent overfitting and ensure the model generalizes well to new data.

"""

for idx, batch_size in enumerate([8, 16, 32, 64, 128]):
    history = model.fit(train_datagen.flow(x_train, y_train,
                                         batch_size=10),
                    epochs=n_epoch,
                    callbacks=[reduce_lr,early_stop],
                    validation_data=(x_valid,y_valid))
   
    # Plotting the results on Graph

    fig, ax = plt.subplots(2,1)
    ax[0].plot(history.history['loss'], color='b', label="Training loss")
    ax[0].plot(history.history['val_loss'], color='r', label="validation loss")
    legend = ax[0].legend(loc='best', shadow=True)

    ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
    ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
    legend = ax[1].legend(loc='best', shadow=True)

    plt.show()  # Ensure the plot displays within each iteration

    predictions_model = model.predict(x_test)
    predictions_model_max = np.argmax(predictions_model,axis=1)
    predictions_onehot = to_categorical(predictions_model_max)
    accuracy = accuracy_score(y_test,predictions_onehot)

    print(classification_report(y_test,predictions_onehot,target_names=['Insects','Plants']))

    # Start MLflow
    RUN_NAME = f"run_{idx}"
    with mlflow.start_run(experiment_id=EXPERIMENT_ID, run_name=RUN_NAME) as run:
        # Retrieve run id
        RUN_ID = run.info.run_id
        env_Latest_RUN_ID = RUN_ID
        # Track parameters
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("epochs", 60)
        mlflow.log_param("image_size", 256)


        # Track model
        mlflow.sklearn.log_model(model, "Bat_guts_IVP_classifier")

        #Track confusion matrix results
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric('Weighted Precision',(precision_score(y_test,predictions_onehot, average='weighted')))
        mlflow.log_metric('Weighted Recall',(recall_score(y_test,predictions_onehot, average='weighted')))
        mlflow.log_metric('Weighted F1-score',(f1_score(y_test,predictions_onehot, average='weighted')))


In [ ]:
if env_WANT_NGROK:
    ngrok.kill()

    # Setting the authtoken (optional)
    # Get your authtoken from https://dashboard.ngrok.com/auth
    NGROK_AUTH_TOKEN = "2AVGMje37gfu1yzYm2UJWam40Mm_dmYa8aw82QJvUCwSCETo"
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    # Open an HTTPs tunnel on port 5000 for http://localhost:5000
    ngrok_tunnel = ngrok.connect(port="5000", proto="http", options={"bind_tls": True})
    print("MLflow Tracking UI:", ngrok_tunnel)

In [ ]:
# !mlflow ui --backend-store-uri /content/drive/MyDrive/mlruns

In [ ]:
# Plotting the results on Graph
#fig, ax = plt.subplots(2,1)
#ax[0].plot(history.history['loss'], color='b', label="Training loss")
#ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
#legend = ax[0].legend(loc='best', shadow=True)

#ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
#ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
#legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
# Predictions on testing data created from training data
predictions_model = model.predict(x_test)

In [ ]:
predictions_model_max = np.argmax(predictions_model,axis=1)
predictions_onehot = to_categorical(predictions_model_max)

In [ ]:
y_test

In [ ]:
predictions_onehot

In [ ]:
# Classification Report
print(classification_report(y_test,predictions_onehot,target_names=['Insects','Plants']))

In [ ]:
from keras.applications.xception import preprocess_input,decode_predictions

In [ ]:
import keras

In [ ]:
print(keras.__version__)

In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
def classify(img_path,model):
    img = image.load_img(img_path, target_size=(256, 256))
    img_array = image.img_to_array(img)

    img_batch = np.expand_dims(img_array, axis=0)

    img_preprocessed = preprocess_input(img_batch)

    prediction = model.predict(img_preprocessed)
    print(prediction)
    prediction_max = [int(i > .5) for i in prediction[0]]

    if 1 in prediction_max and sum(prediction_max)==1:
      # if prediction_max.index(1) ==0:
      #   return 'Bat eats Blood'
      if prediction_max.index(1) ==0:
        return 'This Bat eat Insects'
      if prediction_max.index(1) ==1:
        return 'This Bat eat Plants'

    else:
      return prediction_max

In [ ]:
logged_model = "runs:/" + str( env_Latest_RUN_ID) + "/Bat_guts_IVP_classifier"
# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [ ]:
import pickle
import numpy as np

In [ ]:
zBaseDir = env_CONTENT_PATH + "/mlruns/"
zSubDir = get_most_recent_numeric_subdirectory( zBaseDir )
print( "zSubDir="+zSubDir)
zSubSubDir = get_most_recent_numeric_subdirectory( zSubDir )
loaded_model = pickle.load(open( zSubSubDir + "/artifacts/Bat_guts_IVP_classifier/model.pkl", 'rb'))

In [ ]:
zBase= env_CONTENT_PATH + '/images/Bats_mixed_images_RGB_256/Plants'
zFilePath = zBase + "/DRXXX_Brachyphylla_pumila-LY20_13-nectar-guts-LY20-13A-HE-006-Plane004__red.jpeg"
classify(zFilePath,loaded_model)

In [ ]:
zBase= env_CONTENT_PATH + '/images/Bats_mixed_images_RGB_256/Plants'
zFilePath = zBase + "/DRXXX_Brachyphylla_pumila-LY20_13-nectar-guts-LY20-13A-HE-006-Plane005__blue.jpeg"
classify(zFilePath,loaded_model)


In [ ]:
paths = glob.glob(os.path.join(env_CONTENT_PATH + "/images/Bats_mixed_images_RGB_256/"+ str('Plants')+"/","*.jpeg"))


In [ ]:
import pickle

In [ ]:
zBaseDir = env_CONTENT_PATH + "/mlruns/"
zSubDir = get_most_recent_numeric_subdirectory( zBaseDir )
print( "zSubDir="+zSubDir)
zSubSubDir = get_most_recent_numeric_subdirectory( zSubDir )
loaded_model = pickle.load(open( zSubSubDir + "/artifacts/Bat_guts_IVP_classifier/model.pkl", 'rb'))


In [ ]:
len(paths)

In [ ]:
k=0
for path in paths:

  if 'Bat eats Insects' == classify(path,loaded_model):
    print('here')
    k=k+1
  else:
    print(path)



In [ ]:
k,len(paths)

In [ ]:
j=0
for path in paths:

  if 'Bat eats Plants' == classify(path,loaded_model):
    print('here')
    j=j+1
  else:
    print(path)

In [ ]:
j,len(paths)

In [ ]:
#check versions
import flask

In [ ]:
print(flask.__version__)

In [ ]:
print(np.__version__)

In [ ]:
!python3 --version

In [ ]:
import mlflow

In [ ]:
print(mlflow.__version__)